## Validating pre-trained models
This code demonstrates how to load our trained models and one way of doing using them for inference.
It requires that you downloaded and extracted the [pretrained models and the corresponding preprocessed version of kinodata-3D](https://zenodo.org/records/10410594)
in the root directory of this repository.

In [1]:
import json
from pathlib import Path
from typing import Any

import torch

import kinodata.configuration as cfg
from kinodata.model import ComplexTransformer, DTIModel, RegressionModel
from kinodata.model.complex_transformer import make_model as make_complex_transformer
from kinodata.model.dti import make_model as make_dti_baseline
from kinodata.data.data_module import make_kinodata_module
from kinodata.transform import TransformToComplexGraph

/home/raquellrdc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!wandb disabled

W&B disabled.


Demo boilerplate code for loading model checkpoints, reuses parts of our training/evaluation code.

In [3]:
model_dir = Path("..") / "models"
assert model_dir.exists()

def path_to_model(rmsd_threshold: int, split_type: str, split_fold: int, model_type: str) -> Path:
    p = model_dir / f"rmsd_cutoff_{rmsd_threshold}" / split_type / str(split_fold) / model_type
    if not p.exists():
        p.mkdir(parents=True)
    return p
model_cls = {
    "DTI": make_dti_baseline,
    "CGNN": make_complex_transformer,
    "CGNN-3D": make_complex_transformer
}

def load_wandb_config(
    config_file: Path
) -> dict[str, Any]:
    with open(config_file, "r") as f_config:
        config = json.load(f_config)
    config = {str(key): value["value"] for key, value in config.items()}
    return config

def load_from_checkpoint(
    rmsd_threshold: int,
    split_type: str,
    fold: int,
    model_type: str
) -> RegressionModel:
    cls = model_cls[model_type]
    p = path_to_model(rmsd_threshold, split_type, fold, model_type)
    model_ckpt = list(p.glob("**/*.ckpt"))[0]
    model_config = p / "config.json"
    ckp = torch.load(model_ckpt, map_location="cpu")
    config = cfg.Config(load_wandb_config(model_config))
    model = cls(config)
    assert isinstance(model, RegressionModel)
    model.load_state_dict(ckp["state_dict"])
    return model

Load model checkpoints for *scaffold-split* data subject to predicted RMSD $\leq 4\text{Å}$, where the $0$-th fold is used as test set.

In [4]:
cgnn = load_from_checkpoint(4, "scaffold-k-fold", 0, "CGNN")
cgnn_3d = load_from_checkpoint(4, "scaffold-k-fold", 0, "CGNN-3D") 
dti = load_from_checkpoint(4, "scaffold-k-fold", 0, "DTI")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


RuntimeError: Error(s) in loading state_dict for ComplexTransformer:
	size mismatch for out.3.weight: copying a param with shape torch.Size([1, 256]) from checkpoint, the shape in current model is torch.Size([3, 256]).
	size mismatch for out.3.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([3]).

Create the matching data module

In [5]:
data_module = make_kinodata_module(
    cfg.get("data", "training").update(
        dict(
            batch_size=32,
            split_type="scaffold-k-fold",
            filter_rmsd_max_value=4.0,
            split_index=0,
        )
    ),
    transforms=[TransformToComplexGraph(remove_heterogeneous_representation=False)],
)

Processing...


Entering make_data_list
succesfully runing davids funciton
File exists
Reading data frame from /home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/raw/posit_combined.sdf...
Retrieving file names
Retrieving ligand id
Retrieving kinase pdb id
Retrieving fingerprint similarity
Retrieving predicted RMSD
Retrieving KLIFS_structure_ID
Retrieving UniprotID


100%|██████████| 589/589 [02:08<00:00,  4.58it/s]


Adding pocket sequences...
Checking for missing pocket mol2 files...


100%|██████████| 589/589 [00:00<00:00, 28602.38it/s]


...

Exiting with 3552 cached sequences.
(119713, 26)


100%|██████████| 100/100 [00:01<00:00, 94.80it/s]


Exiting make_data_list
Applying pre filter..
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/kinodata/kinodata_docked_v2.pt
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data
['kinodata_docked_v2.pt']
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/kinodata/filter_predicted_rmsd_le4.00/kinodata_docked_v2.pt
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data
['kinodata_docked_v2.pt']
Creating data module for kinodataset:
    split:Split[int64](train=34, val=4, test=5, source=/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/kinodata/filter_predicted_rmsd_le4.00/scaffold-k-fold/1:5.csv)
    train_transform:Compose([
  TransformToComplexGraph()
])
    val_transform:Compose([
  TransformToComplexGraph()
])


Done!
Processing...
Done!
Processing...
Done!


/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/kinodata/filter_predicted_rmsd_le4.00/kinodata_docked_v2.pt
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data
['kinodata_docked_v2.pt']
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/kinodata/filter_predicted_rmsd_le4.00/kinodata_docked_v2.pt
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data
['kinodata_docked_v2.pt']
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/kinodata/filter_predicted_rmsd_le4.00/kinodata_docked_v2.pt
/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data
['kinodata_docked_v2.pt']
Creating data module for davidsdockeddataset:
    split:Split[int64](train=27, val=5, test=5, source=/home/raquellrdc/Desktop/postdoc/fast_ml/kinodata-3D-affinity-prediction/data/processed/davidsdocked/filter_predicted_rmsd_le4.00/scaffold-k-fo

Fast way of demonstrating inference on just one test batch:

In [ ]:
demo_test_batch = next(iter(data_module.test_dataloader()))
with torch.no_grad():
    mae_sample = {
        "DTI sample test MAE": dti.test_step(demo_test_batch)["test/mae"],
        "CGNN sample test MAE": cgnn.test_step(demo_test_batch)["test/mae"],
        "CGNN-3D sample test MAE": cgnn_3d.test_step(demo_test_batch)["test/mae"],
    }
mae_sample

/Users/joschka/mambaforge/envs/kinodata/lib/python3.10/site-packages/pytorch_lightning/core/module.py:377: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


{'DTI sample test MAE': tensor(0.9323),
 'CGNN sample test MAE': tensor(0.9237),
 'CGNN-3D sample test MAE': tensor(0.9096)}

Test all three models using all test data in the current data module

In [ ]:
from pytorch_lightning import Trainer

In [ ]:
trainer = Trainer(logger=False)
dti_metrics = trainer.test(model=dti, datamodule=data_module, ckpt_path=None)
cgnn_metrics = trainer.test(model=cgnn, datamodule=data_module, ckpt_path=None)
cgnn_3d_metrics = trainer.test(model=cgnn_3d, datamodule=data_module, ckpt_path=None)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/joschka/mambaforge/envs/kinodata/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 136/136 [00:08<00:00, 16.55it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/corr           0.5125585198402405
        test/mae            0.8224371671676636
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 136/136 [02:07<00:00,  1.07it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/corr           0.4546387493610382
        test/mae      

Display results

In [ ]:
{
    "DTI": dti_metrics, 
    "CGNN": cgnn_metrics, 
    "CGNN-3D": cgnn_3d_metrics
}

{'DTI': [{'test/mae': 0.8224371671676636, 'test/corr': 0.5125585198402405}],
 'CGNN': [{'test/mae': 0.8519962430000305, 'test/corr': 0.4546387493610382}],
 'CGNN-3D': [{'test/mae': 0.7448561787605286, 'test/corr': 0.632169246673584}]}